<a href="https://colab.research.google.com/github/Hayou-Azizkd/AI_Dossier_Prioritisation_PoC/blob/main/Notebook_PoC_Agent_IA_de_priorisation_de_dossiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent IA d’aide à la priorisation de dossiers — PoC

**But du notebook :** démontrer une logique de priorisation **indicative** de dossiers administratifs
(texte + métadonnées), avec une **justification** et une validation **human-in-the-loop**.

> Ce notebook illustre la faisabilité et la transparence de l’approche, pas une performance de production.


## Données (PoC)
Nous utilisons un petit échantillon de dossiers **synthétiques** avec :
- `description` (texte libre)
- `date_soumission`, `categorie`, `urgence_declaree`

Objectif : tester la logique de priorisation et la capacité d’explication.


In [ ]:
import pandas as pd
from datetime import datetime

data = [
    {"id":"D-001","date_soumission":"2026-01-02","categorie":"Aide financière","urgence_declaree":"Élevée",
     "description":"Je risque une coupure de service cette semaine, pas de réponse depuis 10 jours."},
    {"id":"D-002","date_soumission":"2026-01-03","categorie":"Information générale","urgence_declaree":"Faible",
     "description":"Je souhaite connaître les documents requis pour déposer une demande."},
    {"id":"D-003","date_soumission":"2025-12-20","categorie":"Plainte","urgence_declaree":"Moyenne",
     "description":"Mon dossier est bloqué depuis un mois et je n'arrive pas à obtenir une mise à jour."},
    {"id":"D-004","date_soumission":"2026-01-01","categorie":"Changement de situation","urgence_declaree":"Élevée",
     "description":"Je viens de perdre mon emploi et j'ai besoin de mettre à jour ma situation rapidement."},
    {"id":"D-005","date_soumission":"2025-12-28","categorie":"Aide financière","urgence_declaree":"Moyenne",
     "description":"Je n'ai pas reçu le paiement prévu et je ne sais pas quoi faire."},
    {"id":"D-006","date_soumission":"2026-01-04","categorie":"Information générale","urgence_declaree":"Faible",
     "description":"Pouvez-vous confirmer la réception de mon dossier ?"},
]

df = pd.DataFrame(data)
df


,id,date_soumission,categorie,urgence_declaree,description
0,D-001,2026-01-02,Aide financière,Élevée,Je risque une coupure de service cette semaine...
1,D-002,2026-01-03,Information générale,Faible,Je souhaite connaître les documents requis pou...
2,D-003,2025-12-20,Plainte,Moyenne,Mon dossier est bloqué depuis un mois et je n'...
3,D-004,2026-01-01,Changement de situation,Élevée,Je viens de perdre mon emploi et j'ai besoin d...
4,D-005,2025-12-28,Aide financière,Moyenne,Je n'ai pas reçu le paiement prévu et je ne sa...
5,D-006,2026-01-04,Information générale,Faible,Pouvez-vous confirmer la réception de mon doss...


## Préparation
On crée des variables simples et auditable :
- `jours_depuis_soumission` : ancienneté du dossier
- `urgence_score` : encodage numérique de l’urgence déclarée (Faible/Moyenne/Élevée)

Ces transformations sont volontairement simples pour garantir la transparence.


In [ ]:
def days_since(date_str: str, ref_date: str = "2026-01-04") -> int:
    d = datetime.strptime(date_str, "%Y-%m-%d")
    ref = datetime.strptime(ref_date, "%Y-%m-%d")
    return (ref - d).days

df["jours_depuis_soumission"] = df["date_soumission"].apply(days_since)

urgence_map = {"Faible": 0, "Moyenne": 1, "Élevée": 2}
df["urgence_score"] = df["urgence_declaree"].map(urgence_map).fillna(0).astype(int)

df[["id","date_soumission","jours_depuis_soumission","categorie","urgence_declaree","urgence_score"]]


,id,date_soumission,jours_depuis_soumission,categorie,urgence_declaree,urgence_score
0,D-001,2026-01-02,2,Aide financière,Élevée,2
1,D-002,2026-01-03,1,Information générale,Faible,0
2,D-003,2025-12-20,15,Plainte,Moyenne,1
3,D-004,2026-01-01,3,Changement de situation,Élevée,2
4,D-005,2025-12-28,7,Aide financière,Moyenne,1
5,D-006,2026-01-04,0,Information générale,Faible,0


## Signal textuel (approche simple)
On détecte quelques mots-clés indicatifs (ex. “bloqué”, “coupure”, “sans réponse”).
Ce signal est **faible** et sert uniquement à enrichir la priorisation dans un cadre PoC.


## Scoring transparent
Le score combine 3 familles de signaux :
1) Urgence déclarée (poids principal)
2) Ancienneté du dossier (éviter qu’un dossier reste ignoré)
3) Signal textuel (mots-clés) — utilisé avec prudence

Le résultat reste **indicatif** et doit être validé par un humain.


In [ ]:
KEYWORDS = ["coupure", "perdre", "bloqué", "sans réponse", "urgence", "rapidement", "paiement"]

def keyword_hit(text: str) -> bool:
    t = str(text).lower()
    return any(k in t for k in KEYWORDS)

df["keyword_signal"] = df["description"].apply(keyword_hit).astype(int)

def score_row(row) -> int:
    score = 0

    # 1) Urgence déclarée (poids principal)
    score += row["urgence_score"] * 3

    # 2) Ancienneté (plus c'est ancien, plus ça remonte)
    if row["jours_depuis_soumission"] >= 10:
        score += 2
    elif row["jours_depuis_soumission"] >= 5:
        score += 1

    # 3) Signal texte (faible mais utile en PoC)
    if row["keyword_signal"] == 1:
        score += 2

    return score

df["score"] = df.apply(score_row, axis=1)

def priority_label(score: int) -> str:
    if score >= 7:
        return "Élevée"
    if score >= 4:
        return "Moyenne"
    return "Faible"

df["priorite_suggeree"] = df["score"].apply(priority_label)
df.sort_values("score", ascending=False)[["id","categorie","urgence_declaree","jours_depuis_soumission","keyword_signal","score","priorite_suggeree"]]


,id,categorie,urgence_declaree,jours_depuis_soumission,keyword_signal,score,priorite_suggeree
0,D-001,Aide financière,Élevée,2,1,8,Élevée
3,D-004,Changement de situation,Élevée,3,1,8,Élevée
2,D-003,Plainte,Moyenne,15,1,7,Élevée
4,D-005,Aide financière,Moyenne,7,1,6,Moyenne
1,D-002,Information générale,Faible,1,0,0,Faible
5,D-006,Information générale,Faible,0,0,0,Faible


## Justification (explicabilité)
Pour chaque dossier, on génère une justification courte qui explique **quels facteurs** ont contribué à la
priorité suggérée. L’objectif est de rendre la recommandation compréhensible par un non-spécialiste.


In [ ]:
def build_justification(row) -> str:
    parts = [
        f"Urgence déclarée: {row['urgence_declaree']}.",
        f"Jours depuis soumission: {row['jours_depuis_soumission']}."
    ]
    if row["keyword_signal"] == 1:
        parts.append("Signaux textuels détectés (mots-clés liés à un blocage ou une situation urgente).")
    parts.append("Priorisation indicative — validation humaine requise.")
    return " ".join(parts)

df["justification"] = df.apply(build_justification, axis=1)
df.sort_values("score", ascending=False)[["id","priorite_suggeree","justification"]]


,id,priorite_suggeree,justification
0,D-001,Élevée,Urgence déclarée: Élevée. Jours depuis soumiss...
3,D-004,Élevée,Urgence déclarée: Élevée. Jours depuis soumiss...
2,D-003,Élevée,Urgence déclarée: Moyenne. Jours depuis soumis...
4,D-005,Moyenne,Urgence déclarée: Moyenne. Jours depuis soumis...
1,D-002,Faible,Urgence déclarée: Faible. Jours depuis soumiss...
5,D-006,Faible,Urgence déclarée: Faible. Jours depuis soumiss...


## Human-in-the-loop (validation)
Dans un usage réel, un agent humain :
- accepte, corrige ou rejette la priorité suggérée,
- ajoute un commentaire de contexte.

On simule ici cette étape pour illustrer le positionnement “aide à la décision”.


In [ ]:
# Simulation : l'humain peut accepter ou corriger la suggestion
df["validation_humaine"] = "À valider"
df["commentaire"] = ""

df.loc[df["id"]=="D-002", ["validation_humaine","commentaire"]] = ["Corrigée", "Cas simple, pas urgent malgré la date."]
df.loc[df["id"]=="D-001", ["validation_humaine","commentaire"]] = ["Acceptée", "Risque de coupure mentionné."]

df.sort_values("score", ascending=False)[["id","priorite_suggeree","validation_humaine","commentaire"]]


,id,priorite_suggeree,validation_humaine,commentaire
0,D-001,Élevée,Acceptée,Risque de coupure mentionné.
3,D-004,Élevée,À valider,
2,D-003,Élevée,À valider,
4,D-005,Moyenne,À valider,
1,D-002,Faible,Corrigée,"Cas simple, pas urgent malgré la date."
5,D-006,Faible,À valider,


## Limites (PoC)
- Données synthétiques : résultats illustratifs, non généralisables.
- Dépendance à l’urgence déclarée et aux mots-clés (risque de faux positifs/négatifs).
- Le système ne doit pas être utilisé sans validation humaine.
